In [1]:
import os
import json
import cobra
import pandas as pd
import time
import re
from goatools import obo_parser

import sys
sys.path.append('../script')
from GO_Kcat_analysis import *

# Input and output data

In [ ]:
# Model file
model_file = '../data/iML1515R/iML1515R.json'

# GO-kcat relationship data from UniProt
GO_kcat_file = '../data/GO/GO_kcat_tree_total.csv'

#download an .obo file of the most current GO: 
# wget http://current.geneontology.org/ontology/go-basic.obo
obo_path = '../data/GO/go-basic.obo'

# The Taxonomy information corresponding to the model's species.
org_name = 'Corynebacterium glutamicum'  # replace with the actual biological name ：Bacillus subtilis，Thermus thermophilus，Corynebacterium glutamicum
org_type = 'org_name' 

# Output file
model_name = os.path.splitext(os.path.basename(model_file))[0]
output_dir = f"../analysis/DeepGO-SE/{model_name}"
os.makedirs(output_dir, exist_ok=True)

#model gene information
gene_csv_path = os.path.join(output_dir, 'gene.csv')

# protein sequences information
raw_fasta_path = os.path.join(output_dir, 'protein_sequences.fasta')

# cleaned protein sequences information
cleaned_fasta_path = os.path.join(output_dir, f"{model_name}_protein_sequences_cleaned.fasta")

# DeepGO-SE/ predicted data
deepgo_base = f"../data/DeepGO-SE/{model_name}"

# Calculate GO-kcat values
go_kcat_out_file = os.path.join(output_dir, "go_kcat_mean_parent.json")

# Get ptrotein sequence

Extract the gene ID from the model, obtain the corresponding protein sequence, and save these sequences as FASTA format files. At the same time, it also cleans the generated FASTA file, removes duplicate sequences, and saves the cleaned result as a new FASTA file.



In [3]:
# 1. Read model
model = cobra.io.load_json_model(model_file)
convert_to_irreversible(model)
norm_model = isoenzyme_split(model)

# 2. Extract gene IDs and save them to a CSV file
gene_ids = [gene.id for gene in norm_model.genes]
gene_df = pd.DataFrame({'gene': gene_ids})
gene_df.to_csv(gene_csv_path, index=False)
print(f"gene.csv has been saved to: {gene_csv_path}")

# 3. Retrieve protein sequences and write them to a FASTA file (with error retries)

with open(raw_fasta_path, 'w') as output_file:
    for idx, gene_name in enumerate(gene_ids, 1):
        raw_fasta = get_uniprot_sequence_by_gene(gene_name)
        if raw_fasta:
            output_file.write(raw_fasta.strip() + '\n')  
        else:
            print(f"Failed to retrieve sequence for: {gene_name}")

        if idx % 50 == 0:
            print(f"Processed {idx}/{len(gene_ids)} genes")
        time.sleep(0.3)
print(f"Raw FASTA has been saved to: {raw_fasta_path}")

# 4. Clean the FASTA file (remove duplicates)
with open(raw_fasta_path, "r") as file:
    input_text = file.read()

pattern = re.compile(r'^>([^\n]+)\n([^>]*)', re.MULTILINE)
matches = pattern.findall(input_text)

seen_headers = set()
cleaned_lines = []
for header_line, sequence_block in matches:
   
    if '|' in header_line:
        header_id = header_line.split('|')[1]  
    else:
        header_id = header_line.strip()        

    if header_id not in seen_headers:
        seen_headers.add(header_id)

        sequence = sequence_block.replace('\n', '').replace(' ', '')
        formatted_seq = '\n'.join([sequence[i:i+60] for i in range(0, len(sequence), 60)])
        cleaned_lines.append(f'>{header_id}\n{formatted_seq}\n')

with open(cleaned_fasta_path, "w") as file:
    file.writelines(cleaned_lines)

print(f"Cleaning completed. Results saved to: {cleaned_fasta_path}")
print(f"Total unique sequences: {len(seen_headers)}")

gene.csv has been saved to: ../analysis/DeepGO-SE/iML1515R/gene.csv
Processed 50/1516 genes
Processed 100/1516 genes
Processed 150/1516 genes
Processed 200/1516 genes
Processed 250/1516 genes
Processed 300/1516 genes
Processed 350/1516 genes
Processed 400/1516 genes
Processed 450/1516 genes
Processed 500/1516 genes
Processed 550/1516 genes
Processed 600/1516 genes
Processed 650/1516 genes
Processed 700/1516 genes
Processed 750/1516 genes
Processed 800/1516 genes
Processed 850/1516 genes
Processed 900/1516 genes
Processed 950/1516 genes
Processed 1000/1516 genes
Processed 1050/1516 genes
Processed 1100/1516 genes
Processed 1150/1516 genes
Processed 1200/1516 genes
Processed 1250/1516 genes
Processed 1300/1516 genes
Processed 1350/1516 genes
Processed 1400/1516 genes
Processed 1450/1516 genes
Processed 1500/1516 genes
Raw FASTA has been saved to: ../analysis/DeepGO-SE/iML1515R/protein_sequences.fasta
Cleaning completed. Results saved to: ../analysis/DeepGO-SE/iML1515R/iML1515R_protein_se

# DeepGO-SE prediction

DeepGO-SE was used to predict the GO function of protein sequences in {model_name}_protein_sequences_cleaned, supporting three types of GO: BP, MF, and CC.

Reference: https://www.nature.com/articles/s42256-024-00795-w

Code repository：https://github.com/bio-ontology-research-group/deepgo2


# Integration of GO predicted by DeepGO-SE

Load the DeepGO-SE prediction results, merge the prediction data of different GO types (BP, CC, MF), and count the number of predictions for each GO type.

In [4]:
# Load DeepGO-SE prediction results
go_types = ['BP', 'CC', 'MF']
df_list = []
for go in go_types:
    fp = os.path.join(deepgo_base, f"{go}.csv")
    df = pd.read_csv(fp, header=None, sep='\t', names=['uniprot', 'GO_term', 'kcat'])
    df['go_type'] = go
    df_list.append(df)
df_final = pd.concat(df_list)
df_final.to_excel(os.path.join(deepgo_base, "GO_Predictions_Combined.xlsx"), index=False)

result = {}
for _, row in df_final.iterrows():
    result.setdefault(row['go_type'], {}).setdefault(row['uniprot'], set()).add(row['GO_term'])
for gt in result:
    for uid in result[gt]:
        result[gt][uid] = list(result[gt][uid])
        
print("GO term counts by type:")
print(df_final['go_type'].value_counts())

GO term counts by type:
BP    75631
MF    30511
CC    15317
Name: go_type, dtype: int64


It is to load the GO (Gene Ontology) data and the kcat value file, and calculate the average kcat value of the GO items of the specified organisms (such as Escherichia coli, Bacillus subtilis, etc.)

In [5]:
go = obo_parser.GODag(obo_path)

# Calculate GO-kcat values
go_kcat_out_file = os.path.join(output_dir, "go_kcat_mean_parent.json")
go_term_mean_kcat_dict = get_go_term_mean_kcat_by_org_v3_2(norm_model, result, org_type, org_name, GO_kcat_file, go)
with open(go_kcat_out_file, 'w') as f:
    json.dump(go_term_mean_kcat_dict, f, indent=4)

../data/GO/go-basic.obo: fmt(1.2) rel(2025-03-16) 43,544 Terms


100%|██████████| 5883/5883 [11:14<00:00,  8.72it/s] 


# Obtain the Reaction-gene-GO relationship

In [ ]:
# Post-process into max/mean/median
for go_type in ['Total']:
    for clc_type in ['median']:
        out_file = os.path.join(output_dir, f'go_kcat_mean_parent_process_{go_type}_{clc_type}.json')
        data = process_data(go_term_mean_kcat_dict, go_type, clc_type)
        with open(out_file, 'w') as f:
            json.dump(data, f, indent=4)

print("\nSummary of GO-term kcat coverage:")
for gt in ['Total']:
    count = sum(1 for reac in go_term_mean_kcat_dict if any(go_term_mean_kcat_dict[reac][gt].values()))
    print(f"{gt}: {count} reactions")


Summary of GO-term kcat coverage:
Total: 5338 reactions
